# Продажи недвижимости в России (поиск выбросов)
**Источник:** <br>https://www.kaggle.com/
<br>
<br>
**Цели:** <br>● провести предобработку данных, для последующего их удобного анализа
<br>
● проанализировать данные в соответсвии с имеющимися задачами
<br>
<br>
**Задачи:** <br>● проверить данные на мультиколлинеарность
<br>● исключиь один или несколько признаков 
<br>● преобразуйте категориальные признаки в численные
<br>● получить предсказания модели на валидационной выборке
<br>● сравнить с предсказаниями модели, выдающей среднюю цену
<br>
<br>
**Входные данные:** <br>входные данные представляют собой 2 файла csv, содержащие в себе информацию о недвижимости 2х регионов
<br>
<br>
**Используемые библиотеки:** <br>pandas
<br>numpy
<br>sklearn
<br>matplotlib.pyplot
<br>seaborn
<br>math
<br>sklearn.model_selection
<br>sklearn.linear_model
<br>sklearn.metrics
<br>sklearn.preprocessing
<br>
<br>
Для начала импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler 
import math

Считаем 2 файла, содержащие данные по недвижимости двух регионов

In [2]:
df1 = pd.read_csv("out2.csv")
df2 = pd.read_csv("out3.csv")

Сохраним их в один файл и выведем основую информацию

In [3]:
df = df1.append(df2, ignore_index=True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16609 entries, 0 to 16608
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     16609 non-null  int64  
 1   price          16609 non-null  int64  
 2   region         16609 non-null  int64  
 3   building_type  16609 non-null  int64  
 4   rooms          16609 non-null  int64  
 5   area           16609 non-null  float64
 6   object_type    16609 non-null  int64  
 7   prise_meter    16609 non-null  float64
 8   month          16609 non-null  int64  
 9   year           16609 non-null  int64  
 10  flat           16609 non-null  int64  
dtypes: float64(2), int64(9)
memory usage: 1.4 MB


/var/folders/rt/mvnmln8j70d9fmnh5b6pl1qr0000gn/T/ipykernel_95201/69343806.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2, ignore_index=True)


Почему-то когда я удаляю столбец rooms или area, погрешность увеличивается, так я решила это не делать

In [4]:
df = df.drop(["Unnamed: 0"], axis=1)

Преобразую категориальные признаки в численные:

In [5]:
df["first_flat"] = df["flat"].apply(lambda x: int(x == 0))
df["last_flat"] = df["flat"].apply(lambda x: int(x == 1))
df = df.drop(["flat"], axis = 1)

In [6]:
df["panel"] = df["building_type"].apply(lambda x: int(x == 1))
df["mono"] = df["building_type"].apply(lambda x: int(x == 2))
df["brick"] = df["building_type"].apply(lambda x: int(x == 3))
df["block"] = df["building_type"].apply(lambda x: int(x == 4))
df["wood"] = df["building_type"].apply(lambda x: int(x == 5))
df = df.drop(["building_type"], axis = 1)

In [7]:
df = pd.get_dummies(df, columns=['object_type'], drop_first= True)

In [8]:
df = pd.get_dummies(df, columns=['region'], drop_first= True)

In [9]:
df.head()

,price,rooms,area,prise_meter,month,year,first_flat,last_flat,panel,mono,brick,block,wood,object_type_11,region_5282
0,1800000,2,53.0,33962.264151,9,2018,1,0,0,0,1,0,0,0,1
1,2600000,3,65.6,39634.146341,9,2018,0,0,1,0,0,0,0,1,1
2,1600000,3,62.0,25806.451613,9,2018,0,0,0,0,1,0,0,0,1
3,1400000,1,41.0,34146.341463,9,2018,0,0,0,0,1,0,0,0,1
4,1650000,2,43.0,38372.093023,9,2018,0,0,0,0,1,0,0,0,1


Выделю целевой признак и другие признаки

In [10]:
target = df["price"]
features = df.drop(["price"], axis=1)

Разбью данные на обучающую и валидационную выборки

In [11]:
np.random.seed(31415)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25)

Приведу признаки к одному масштабу.

In [12]:
scaler = StandardScaler() 
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)

Инициализирую модель линейной регрессии, обучу модель на обучающей выборке. Получу предсказания модели на валидационной выборке и посчитаю метрику RMSE на валидационной выборке.

In [13]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
x = mean_squared_error(target_valid, predictions_valid)
print(math.sqrt(x))

228959.96692164993


Проверю модель на адекатность.

In [14]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(features_train, target_train)
predictions_valid = dummy_regr.predict(features_valid)
x = mean_squared_error(target_valid, predictions_valid)
print(math.sqrt(x))

967706.501049382


## Вывод
Таким образом, после обработки данных, погрешность уменьшилась почти на 20 тысяч. Это не так много. Но если сравнивать с результатом, который выдала модель DummyRegressor (967706.501049382), то результат достаточно хороший. 